In [ ]:
import pandas as pd
import numpy as np
from PIL import Image, ImageOps
import tensorflow as tf
from tensorflow.keras import preprocessing
import csv
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable
from torchvision import models
from tqdm import tqdm

In [ ]:
import os    
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [ ]:
file_path = "TS_Freq_array.csv"

# Initialize arrays
y_labels = []
x_TS = []

# Read the CSV file line by line
with open(file_path, 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    
    next(csv_reader)
    
    # Extract the first column and construct rows as arrays
    for row in csv_reader:
        y_labels.append(row[0])
        x_TS.append(row[1:])

In [ ]:
x_TS = np.array(x_TS, dtype = float)
x_TS.max() #upper limit of plot
x_TS.min() #lower limit of plot

In [ ]:
# Create an array of x values
x_values = np.arange(45,260.5,0.5)
elements_to_remove = [90.0, 90.5, 170.5, 171.0, 171.5, 172.0, 172.5]
x_values = x_values[~np.isin(x_values, elements_to_remove)]
print(x_values)

In [ ]:
from io import BytesIO
Graph_Pixel = []
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
for i in tqdm(range(len(x_TS))):
    virtual_file = BytesIO()
    TS_array = np.array(x_TS[i])
    plt.plot(x_values, TS_array, linestyle='-')
    plt.xlim(45, 260)
    plt.ylim(-133, -5)
    plt.grid(True)
    plt.savefig(virtual_file, format = "png")
    plt.clf()
    plt.close()
    virtual_file.seek(0)
    image = Image.open(virtual_file)
    gray_image = image.convert('L')
    gray_image = gray_image.resize((488, 244), Image.LANCZOS)
    im_as_arr = np.float32(gray_image) # Q: will the axis labels affect the values?
    im_as_arr = np.expand_dims(im_as_arr, axis=2)
    im_as_arr = im_as_arr.transpose(2, 1, 0)
    for channel, _ in enumerate(im_as_arr):
        im_as_arr[channel] /= 255
        im_as_arr[channel] -= mean[channel]
        im_as_arr[channel] /= std[channel]
    Graph_Pixel.append(im_as_arr)

np.savez('/1D CNN/Graph_Pixel.npz', *Graph_Pixel)